In [1]:
import pandas as pd
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment, Border, Side

import solara
import ipywidgets as ipw
from io import BytesIO

In [4]:
def read_button_excel(button):
    return pd.read_excel(BytesIO(button.value[0]["content"]))

def get_ticksheet():
    students = read_button_excel(upload_students)
    students = students[["Apellido(s)","Nombre","Grupo"]]

    df = read_button_excel(upload_encuesta)
    df = df[["Apellido(s)","Nombre","Consulta"]]
    df["Apellido(s)"] = df["Apellido(s)"].str.strip()
    df["Nombre"] = df["Nombre"].str.strip()
    df.insert(3,"val","X")
    df = df.pivot(index=["Apellido(s)","Nombre"], columns='Consulta', values="val").reset_index()
    
    assert set(df["Apellido(s)"]) - set(students["Apellido(s)"]) - set(["Merino Troncoso"]) == set()
    
    df = pd.merge(students,df,"left").fillna("")
    
    side_thin = Side(border_style="thin")
    side_med = Side(border_style="medium")

    outbytes = BytesIO()
    with pd.ExcelWriter(outbytes) as writer:
        for grupo in df.Grupo.unique():
            gdf = df[df.Grupo==grupo].drop("Grupo",axis=1)
            gdf.to_excel(writer,index=False,sheet_name=str(grupo))
    
            # adjust column widths and horizontal alignments
            worksheet = writer.sheets[str(grupo)]
            for idx, col in enumerate(gdf):
                max_len = max((gdf[col].astype(str).map(len).max(), len(col))) + 1
                col_lett = get_column_letter(idx+1)
                worksheet.column_dimensions[col_lett].width = max_len
                worksheet["A1"].alignment = Alignment(horizontal='left')
                worksheet["B1"].alignment = Alignment(horizontal='left')
                for irow, cell in enumerate(worksheet[col_lett]):
                    if idx>=2: # ticks
                        cell.alignment = Alignment(horizontal='center')
                    if irow==0:
                        cell.border = Border(left=side_thin,right=side_thin,top=side_thin,bottom=side_med)
                    elif irow==1:
                        cell.border = Border(left=side_thin,right=side_thin,top=side_med,bottom=side_thin)
                    else:
                        cell.border = Border(left=side_thin,right=side_thin,top=side_thin,bottom=side_thin)
            worksheet.print_area = f'A1:{col_lett}{irow+1}'
    return outbytes.getvalue()

In [5]:
upload_students = ipw.FileUpload(description="Upload students",layout=dict(width='200px'))
upload_encuesta = ipw.FileUpload(description="Upload encuesta",layout=dict(width='200px'))
download = solara.FileDownload(data=get_ticksheet, label="Download ticksheet", filename="ticksheet.xlsx")
display(upload_students, upload_encuesta, download)

FileUpload(value=(), description='Upload students', layout=Layout(width='200px'))

FileUpload(value=(), description='Upload encuesta', layout=Layout(width='200px'))

Cannot show ipywidgets in text